In [10]:
import numpy as np
import openmc
import openmc.deplete
from pyne import serpent
import re
from openmc.data import zam, gnds_name, atomic_mass
from openmc.deplete.coupled_operator import _get_nuclides_with_data
from openmc.deplete import MicroXS

In [16]:
og_mats = openmc.Materials.from_xml('../../saltproc/examples/msbr/materials.xml')
geometry = openmc.Geometry.from_xml('../../saltproc/examples/msbr/geometry.xml', og_mats)
model = openmc.Model(geometry=geometry, materials = og_mats)

In [23]:
rxns, diluted_mats = MicroXS._add_dilute_nuclides('../../saltproc/examples/msbr/chain_endfb71_pwr.xml', model, 1.0e3)

In [32]:
fuel = diluted_mats[0]
percents = []
nucs = []
at_mass = []
for nuc, pt, tp in fuel.nuclides:
    nucs.append(nuc)
    percents.append(pt)
    at_mass.append(atomic_mass(nuc))

percents = np.array(percents)
at_mass = np.array(at_mass)
serp_spec = []
mass_percents = percents*at_mass / np.dot(percents, at_mass)
for name, mass_percent in zip(nucs, mass_percents):
    z, a, m = zam(name)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    serp_zam = ''.join([serp_zam, '.82c'])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')

In [33]:
serp_spec

['            3007.82c  -0.07874746738790854\n',
 '            4009.82c  -0.02255668791383212\n',
 '            9019.82c  -0.4540030121792843\n',
 '           90232.82c  -0.4355791304823362\n',
 '           92233.82c  -0.009113702036638935\n',
 '            1001.82c  -4.995620412962261e-22\n',
 '            1002.82c  -9.983566230967948e-22\n',
 '            1003.82c  -1.4950052716876012e-21\n',
 '            2003.82c  -1.494995378171055e-21\n',
 '            2004.82c  -1.984023604143294e-21\n',
 '            3006.82c  -2.981595984600735e-21\n',
 '            5010.82c  -4.963245622834659e-21\n',
 '            5011.82c  -5.457128755397569e-21\n',
 '            7014.82c  -6.941089983457803e-21\n',
 '            7015.82c  -7.435303533563996e-21\n',
 '            8016.82c  -7.928412119818305e-21\n',
 '            8017.82c  -8.426185787698666e-21\n',
 '           11022.82c  -1.0902275464868303e-20\n',
 '           11023.82c  -1.1395644787460064e-20\n',
 '           12024.82c  -1.188898470220

In [41]:
serp_spec = []
for name, tup in mat[2].get_nuclide_densities().items():
    name, mass_percent, percent_type = tup
    z, a, m = zam(name)
    serp_zam = str(z*1000 + a + (300*int(bool(m)) + 100*m))
    if name in nuclides_with_data:
        serp_zam = ''.join([serp_zam, '.82c'])
    else:
        serp_zam = ''.join([serp_zam, str(m)])
    serp_space = ''.join([" "]*(20 - len(serp_zam)))
    serp_zam = serp_space + serp_zam
    serp_spec.append(serp_zam + "  -" + str(mass_percent) + '\n')

In [34]:
with open('serp_spec.txt', 'w') as f:
    f.writelines(serp_spec)